# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///test.db')
df = pd.read_sql_table('Messages', engine)
X = df['message']
Y = df.iloc[:,4:]

category_names = list(np.array(Y.columns))

### 2. Write a tokenization function to process your text data

In [3]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stemmer= PorterStemmer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = stemmer.stem(lemmatizer.lemmatize(tok).lower().strip())
        if clean_tok not in stopwords.words('english'): clean_tokens.append(clean_tok) 
    
    return clean_tokens

In [5]:
# Test to see tokenize output
print(tokenize(X[0]))

['weather', 'updat', '-', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

np.random.seed(99)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def display_performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))


In [9]:
display_performance(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.62      0.40      0.49      1563
          1       0.83      0.92      0.87      4948
          2       0.73      0.19      0.30        43

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5421
          1       0.81      0.40      0.53      1133

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3829
          1       0.76      0.58      0.65      2725

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {'vect__min_df': [1,2],
              'tfidf__use_idf':[True],
              'clf__estimator__n_estimators':[10,30], 
              'clf__estimator__min_samples_split':[2,5]
             }


# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
np.random.seed(99)
cv.fit(X_train, y_train)
cv.best_params_


{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 30,
 'tfidf__use_idf': True,
 'vect__min_df': 2}

In [12]:
display_performance(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.68      0.36      0.47      1563
          1       0.82      0.95      0.88      4948
          2       0.36      0.19      0.25        43

avg / total       0.78      0.80      0.78      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5421
          1       0.84      0.47      0.61      1133

avg / total       0.89      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.80      0.84      0.82      3829
          1       0.76      0.70      0.73      2725

avg / total       0.78      0.78      0.78      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [14]:
pipeline2.fit(X_train, y_train)
display_performance(pipeline2, X_test, y_test)

related
             precision    recall  f1-score   support

          0       1.00      0.00      0.00      1563
          1       0.76      1.00      0.86      4948
          2       0.43      0.07      0.12        43

avg / total       0.81      0.76      0.65      6554

request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5421
          1       0.33      0.00      0.00      1133

avg / total       0.74      0.83      0.75      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6516
          1       0.00      0.00      0.00        38

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.59      0.99      0.74      3829
          1       0.53      0.01      0.02      2725

avg / total       0.56      0.58      0.44      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [15]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.